In [ ]:
#Town Population Data

cromwell_pop = 7010
masterton_pop = 22400
invercargill_pop = 49800
reefton_pop = 930
#https://nzdotstat.stats.govt.nz/wbos/Index.aspx?

cromwell_area = 15.63 #km²
masterton_area = 22.5 #km²
invercargill_area = 62.95 #km²
reefton_area = 2.63 #km²

cromwell_density = cromwell_pop/cromwell_area
masterton_density = masterton_pop/masterton_area
invercargill_density = invercargill_pop/invercargill_area
reefton_density = reefton_pop/reefton_area

#https://statsnz.maps.arcgis.com/apps/webappviewer/index.html?id=6f49867abe464f86ac7526552fe19787

# Population density for each town
town_density = {
    'Invercargill': invercargill_density,
    'Cromwell': cromwell_density,
    'Masterton': masterton_density,
    'Reefton': reefton_density
}


# Print population densities
print("Population Density of Cromwell:", cromwell_density, "people/km²")
print("Population Density of Masterton:", masterton_density, "people/km²")
print("Population Density of Invercargill:", invercargill_density, "people/km²")
print("Population Density of Reefton:", reefton_density, "people/km²")


Population Density of Cromwell: 448.49648112603967 people/km²
Population Density of Masterton: 995.5555555555555 people/km²
Population Density of Invercargill: 791.1040508339952 people/km²
Population Density of Reefton: 353.61216730038024 people/km²


In [ ]:
#Town Altitude
#https://en-nz.topographic-map.com

cromwell_altitude = 396
masterton_altitude = 111
invercargill_altitude = 18
reefton_altitude = 195

town_altitude = {
    'Invercargill': invercargill_altitude,
    'Cromwell': cromwell_altitude,
    'Masterton': masterton_altitude,
    'Reefton': reefton_altitude
}

In [ ]:
#Wood Burners
invercargill_woodburner=10503
masterton_woodburner=6648
reefton_woodburner=477
cromwell_woodburner=1350

cromwell_area = 15.63 #km²
masterton_area = 22.5 #km²
invercargill_area = 62.95 #km²
reefton_area = 2.63 #km²

invercargill_woodburner_density = invercargill_woodburner/invercargill_area
masterton_woodburner_density = masterton_woodburner/masterton_area
cromwell_woodburner_density = cromwell_woodburner/cromwell_area
reefton_woodburner_density = reefton_woodburner/reefton_area

town_woodburner = {
    'Invercargill': invercargill_woodburner_density,
    'Cromwell': cromwell_woodburner_density,
    'Masterton': masterton_woodburner_density,
    'Reefton': reefton_woodburner_density
}

In [ ]:
print(town_woodburner)

{'Invercargill': 166.84670373312153, 'Cromwell': 86.37236084452975, 'Masterton': 295.46666666666664, 'Reefton': 181.36882129277566}


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

town_names = ['Invercargill', 'Cromwell', 'Masterton', 'Reefton']
file_path_template = '/content/drive/Shareddrives/700Docs/MetaData/Plot/{}_{}.csv'

town_wind = {}
town_pressure = {}

# Load the data for each town
for town in town_names:
    file_path = file_path_template.format(town, "Wind")
    df = pd.read_csv(file_path)
    town_wind[town] = df['Speed(m/s)'].dropna()

    # Normalize wind speed data using Min-Max normalization to range [0, 1]
    scaler_wind = MinMaxScaler()
    town_wind[town] = scaler_wind.fit_transform(town_wind[town].values.reshape(-1, 1)).flatten()

    file_path = file_path_template.format(town, "Pressure")
    df = pd.read_csv(file_path)
    town_pressure[town] = df['Pstn(hPa)'].dropna()

    # Normalize pressure data using Min-Max normalization to range [0, 1]
    scaler_pressure = MinMaxScaler()
    town_pressure[town] = scaler_pressure.fit_transform(town_pressure[town].values.reshape(-1, 1)).flatten()

In [ ]:
from scipy.stats import wasserstein_distance

def earthmover_distance(data1, data2):
    # Compute the Earth Mover's distance (Wasserstein distance) between two distributions
    emd = wasserstein_distance(data1, data2)
    return emd


In [ ]:
from scipy.stats import ks_2samp

def compare_distributions_KS(data1, data2):
    # Perform the Kolmogorov-Smirnov (KS) test between two distributions
    ks_statistic, p_value = ks_2samp(data1, data2)

    return ks_statistic, p_value

In [ ]:
#Original but realised it was wrong
# def normalize_value(value):
#     """
#     Normalize a given value using the specified formula.
#     """
#     return value / 10**np.ceil(np.log10(abs(value)))

In [ ]:
def difference(town1,town2,maxValue,minValue):
    """
    Normalize a given value using decimal scaling
    """
    return abs(abs(town1-town2)-minValue)/(maxValue-minValue)

In [ ]:
import numpy as np

def calculate_town_rankings(town_names, town_wind, town_pressure, town_density, town_altitude,town_woodburner,
                            wind_weight, pressure_weight, density_weight, altitude_weight,woodburner_weight):
    results = {}

    for target_town in town_names:
        scores = {
            source_town: (
                wind_weight * earthmover_distance(town_wind[source_town], town_wind[target_town]) +
                pressure_weight * earthmover_distance(town_pressure[source_town], town_pressure[target_town]) +
                density_weight * difference(town_density[source_town],town_density[target_town],max(town_density.values()),min(town_density.values())) +
                altitude_weight * difference(town_altitude[source_town],town_altitude[target_town],max(town_altitude.values()),min(town_altitude.values())) +
                woodburner_weight * difference(town_woodburner[source_town],town_woodburner[target_town],max(town_woodburner.values()),min(town_woodburner.values()))
            )
            for source_town in town_names if source_town != target_town
        }

        results[target_town] = sorted(scores.items(), key=lambda x: x[1])

    return results



In [ ]:
earthmover_distance(town_wind["Invercargill"], town_wind["Cromwell"])

0.009801594005568952

In [ ]:
wind_weight = 0.3125
pressure_weight = 0.21875
density_weight = 0.3125
altitude_weight = 0.03125
woodburner_weight = 0.125

result_rankings = calculate_town_rankings(town_names, town_wind, town_pressure, town_density, town_altitude,town_woodburner,
                                              wind_weight, pressure_weight, density_weight, altitude_weight, woodburner_weight)

for target_town, rankings in result_rankings.items():
    print(f"Ranking of source towns for target town '{target_town}':")
    for rank, (source_town, score) in enumerate(rankings, start=1):
        print(f"{rank}. {source_town} (Score: {score:.4f})")
    print("\n")

Ranking of source towns for target town 'Invercargill':
1. Cromwell (Score: 0.0440)
2. Reefton (Score: 0.1094)
3. Masterton (Score: 0.1307)


Ranking of source towns for target town 'Cromwell':
1. Invercargill (Score: 0.0440)
2. Reefton (Score: 0.1582)
3. Masterton (Score: 0.2184)


Ranking of source towns for target town 'Masterton':
1. Invercargill (Score: 0.1307)
2. Reefton (Score: 0.1963)
3. Cromwell (Score: 0.2184)


Ranking of source towns for target town 'Reefton':
1. Invercargill (Score: 0.1094)
2. Cromwell (Score: 0.1582)
3. Masterton (Score: 0.1963)




In [ ]:
ground_truth = {'Invercargill': ['Cromwell', 'Reefton', 'Masterton'],
                'Cromwell': ['Invercargill', 'Reefton', 'Masterton'],
                'Masterton': ['Invercargill', 'Cromwell', 'Reefton'],
                'Reefton': ['Invercargill', 'Cromwell', 'Masterton']}


In [ ]:
weight_set_rankings = {}
for weight_idx in range(5):
    current_weight_id = ""
    wind_weight = 0
    pressure_weight = 0
    density_weight = 0
    altitude_weight = 0
    woodburner_weight = 0

    if weight_idx == 0:
        current_weight_id = "Wind"
        wind_weight = 1
    elif weight_idx == 1:
        current_weight_id = "Pressure"
        pressure_weight = 1
    elif weight_idx == 2:
        current_weight_id = "Population Density"
        density_weight = 1
    elif weight_idx ==3:
        current_weight_id = "Altitude"
        altitude_weight = 1
    else:
        current_weight_id = "Wood Burner"
        woodburner_weight = 1

    result_rankings = calculate_town_rankings(town_names, town_wind, town_pressure, town_density, town_altitude,town_woodburner,
                                              wind_weight, pressure_weight, density_weight, altitude_weight, woodburner_weight)

    weight_set_rankings[current_weight_id] = result_rankings

# Print the stored rankings for each weight set
for weight_set, result_rankings in weight_set_rankings.items():
    print(f"Testing {weight_set}:")
    for target_town, source_rankings in result_rankings.items():
        print(f"Ranking of source towns for target town '{target_town}':")
        for rank, (source_town, score) in enumerate(source_rankings, start=1):
            print(f"{rank}. {source_town} (Score: {score:.4f})")
        print("\n")

Testing Wind:
Ranking of source towns for target town 'Invercargill':
1. Cromwell (Score: 0.0098)
2. Reefton (Score: 0.0293)
3. Masterton (Score: 0.0754)


Ranking of source towns for target town 'Cromwell':
1. Invercargill (Score: 0.0098)
2. Reefton (Score: 0.0303)
3. Masterton (Score: 0.0752)


Ranking of source towns for target town 'Masterton':
1. Cromwell (Score: 0.0752)
2. Invercargill (Score: 0.0754)
3. Reefton (Score: 0.0895)


Ranking of source towns for target town 'Reefton':
1. Invercargill (Score: 0.0293)
2. Cromwell (Score: 0.0303)
3. Masterton (Score: 0.0895)


Testing Pressure:
Ranking of source towns for target town 'Invercargill':
1. Cromwell (Score: 0.0107)
2. Masterton (Score: 0.0139)
3. Reefton (Score: 0.0151)


Ranking of source towns for target town 'Cromwell':
1. Invercargill (Score: 0.0107)
2. Reefton (Score: 0.0116)
3. Masterton (Score: 0.0242)


Ranking of source towns for target town 'Masterton':
1. Invercargill (Score: 0.0139)
2. Cromwell (Score: 0.0242)
3. 

In [ ]:
def calculateWeight(w1,w2,w3,w4,w5):
  total = w1+w2+w3+w4+w5
  w1_result = w1/total
  w2_result = w2/total
  w3_result = w3/total
  w4_result = w4/total
  w5_result = w5/total

  return([w1_result,w2_result,w3_result,w4_result,w5_result])



In [ ]:
calculateWeight(10,7,10,1,4)

[0.3125, 0.21875, 0.3125, 0.03125, 0.125]

In [ ]:
weights = calculateWeight(10,7,10,1,4)
weight_set_rankings = {}
current_weight_id = "No Wood Burner"
wind_weight = weights[0]
pressure_weight = weights[1]
density_weight = weights[2]
altitude_weight = weights[3]
woodburner_weight = weights[4]


result_rankings = calculate_town_rankings(town_names, town_wind, town_pressure, town_density, town_altitude,town_woodburner,
                                              wind_weight, pressure_weight, density_weight, altitude_weight, woodburner_weight)

weight_set_rankings[current_weight_id] = result_rankings

# Print the stored rankings for each weight set
for weight_set, result_rankings in weight_set_rankings.items():
    print(f"Testing {weight_set}:")
    print(f"Weights, Wind: {wind_weight}, Pressure: {pressure_weight}, Density: {density_weight}, Altitude: {altitude_weight}, Word Burner: {woodburner_weight}")
    for target_town, source_rankings in result_rankings.items():
        print(f"Ranking of source towns for target town '{target_town}':")
        for rank, (source_town, score) in enumerate(source_rankings, start=1):
            print(f"{rank}. {source_town} (Score: {score:.4f})")
        print("\n")

Testing No Wood Burner:
Weights, Wind: 0.3125, Pressure: 0.21875, Density: 0.3125, Altitude: 0.03125, Word Burner: 0.125
Ranking of source towns for target town 'Invercargill':
1. Cromwell (Score: 0.0440)
2. Reefton (Score: 0.1094)
3. Masterton (Score: 0.1307)


Ranking of source towns for target town 'Cromwell':
1. Invercargill (Score: 0.0440)
2. Reefton (Score: 0.1582)
3. Masterton (Score: 0.2184)


Ranking of source towns for target town 'Masterton':
1. Invercargill (Score: 0.1307)
2. Reefton (Score: 0.1963)
3. Cromwell (Score: 0.2184)


Ranking of source towns for target town 'Reefton':
1. Invercargill (Score: 0.1094)
2. Cromwell (Score: 0.1582)
3. Masterton (Score: 0.1963)


